In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time


import src.models.builders as mb
import src.inference.helpers as ih
import src.modelling_o2.o2satffa as o2satffa


plotsdir = "../../../../PlotsBreathe/O2_modelling/"

In [5]:
# df.to_excel(f"{plotsdir}airway_resistance_df.xlsx")
# Read from excel
df = pd.read_excel(f"{plotsdir}airway_resistance_df.xlsx", index_col=0)
df.ID = df.ID.astype(str)

In [7]:
# Select data for id 330
df[df.ID == "330"]

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,Airway Resistance mean from ecFEV1 (%),Airway Resistance mean from FEV1 (%),Airway Resistance mode from ecFEV1 (%),Airway Resistance mode from FEV1 (%)


In [2]:
# df = breathe_data.build_O2_FEV1_df()
# Save to excel
# df.to_excel(plotsdir + "Breathe_O2_FEV1.xlsx", index=False)
# Load from excel
# df = pd.read_excel(plotsdir + "/Breathe_O2_FEV1.xlsx")
# # ID column as type string
# df["ID"] = df["ID"].astype(str)
# # Date Redocrded as datetime
# df["Date Recorded"] = df["Date Recorded"].dt.date
# sanity_checks.data_types(df)

In [3]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125


In [138]:
df.describe()

,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,Airway Resistance mean from ecFEV1 (%),Airway Resistance mean from FEV1 (%),Airway Resistance mode from ecFEV1 (%),Airway Resistance mode from FEV1 (%)
count,20397,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000
mean,2020-08-09 21:07:02.003235840,2.198363,96.966501,2.263423,34.801147,166.247120,3.507772,97.711339,64.659185,62.787907,99.238755,34.741463,36.402355,36.448154,38.181988
min,2019-02-08 00:00:00,0.490000,76.000000,0.500000,18.000000,143.000000,2.213968,96.975001,15.320382,15.013975,77.519654,1.650449,1.650449,0.000000,0.000000
25%,2020-03-06 00:00:00,1.550000,96.000000,1.610000,27.000000,160.000000,2.979444,97.225960,48.198629,46.536607,98.699543,20.800349,22.381259,23.000000,25.000000
50%,2020-09-27 00:00:00,2.030000,97.000000,2.090000,34.000000,166.000000,3.386997,97.989462,62.880920,61.235539,99.767593,34.627932,36.243703,38.000000,39.000000
75%,2021-01-26 00:00:00,2.760000,98.000000,2.830000,41.000000,173.000000,3.987357,98.114941,77.657779,75.719884,100.255981,49.580092,51.019936,52.000000,54.000000
max,2021-07-24 00:00:00,5.260000,100.000000,5.260000,64.000000,193.000000,5.322753,98.340804,149.505350,149.505350,103.026043,82.944964,84.520873,84.000000,85.000000
std,NaN,0.816148,1.649808,0.822063,10.154773,9.151066,0.649323,0.449598,20.301483,20.271387,1.686950,17.647191,17.945759,19.033484,19.180622


## F1 - Airway resistance vs O2 drop

### Infer AR

In [6]:
# Infer airway resistance using the model
for id in df.ID.unique():
    # for id in ["101", "102"]:
    df_for_ID = df[df.ID == id].copy().reset_index()
    print(f"\nRunning for ID {id}, with {len(df_for_ID)} observations")
    # Take one element of the df for ID at column height
    height = df_for_ID.Height[0]
    sex = df_for_ID.Sex[0]
    hfev1_prior = {
        "type": "default",
        "height": height,
        "age": df_for_ID.Age[0],
        "sex": sex,
    }
    ho2sat_prior = {
        "type": "default",
        "height": height,
        "sex": sex,
    }
    tic = time.time()
    (
        # model,
        # inf_alg,
        # HFEV1,
        # ecFEV1,
        # _,
        # _,
        # AR,
        model,
        inf_alg,
        HFEV1,
        ecFEV1,
        AR,
    ) = mb.build_FEV1_O2_point_in_time_model(hfev1_prior, ho2sat_prior)
    print(f"model took {time.time() - tic} seconds to build")

    tic = time.time()
    df_for_ID["ecFEV1 inf mean"] = np.nan
    df_for_ID["FEV1 inf mean"] = np.nan
    df_for_ID["ecFEV1 inf mode"] = np.nan
    df_for_ID["FEV1 inf mode"] = np.nan
    fev1s_tmp = []
    ecfev1s_tmp = []
    for i in range(len(df_for_ID)):
        fev1_obs = df_for_ID.loc[i, "FEV1"]
        ecfev1_obs = df_for_ID.loc[i, "ecFEV1"]

        # FEV1
        if fev1_obs in fev1s_tmp:
            df_for_ID.loc[i, "FEV1 inf mean"] = df_for_ID.loc[
                df_for_ID["FEV1"] == fev1_obs, "FEV1 inf mean"
            ].values[0]
            df_for_ID.loc[i, "FEV1 inf mode"] = df_for_ID.loc[
                df_for_ID["FEV1"] == fev1_obs, "FEV1 inf mode"
            ].values[0]
        else:
            res_ar_for_fev1 = ih.infer(
                inf_alg, [AR], [[ecFEV1, fev1_obs]], show_progress=False
            )
            df_for_ID.loc[i, "FEV1 inf mean"] = np.multiply(
                res_ar_for_fev1.values, AR.bins
            ).sum()
            df_for_ID.loc[i, "FEV1 inf mode"] = AR.bins[
                np.argmax(res_ar_for_fev1.values)
            ]

        # ecFEV1
        if ecfev1_obs in ecfev1s_tmp:
            df_for_ID.loc[i, "ecFEV1 inf mean"] = df_for_ID.loc[
                df_for_ID["ecFEV1"] == ecfev1_obs, "ecFEV1 inf mean"
            ].values[0]
            df_for_ID.loc[i, "ecFEV1 inf mode"] = df_for_ID.loc[
                df_for_ID["ecFEV1"] == ecfev1_obs, "ecFEV1 inf mode"
            ].values[0]
        else:
            res_ar_for_ecfev1 = ih.infer(
                inf_alg, [AR], [[ecFEV1, ecfev1_obs]], show_progress=False
            )
            df_for_ID.loc[i, "ecFEV1 inf mean"] = np.multiply(
                res_ar_for_ecfev1.values, AR.bins
            ).sum()
            df_for_ID.loc[i, "ecFEV1 inf mode"] = AR.bins[
                np.argmax(res_ar_for_ecfev1.values)
            ]

        fev1s_tmp = np.append(fev1s_tmp, fev1_obs)
        ecfev1s_tmp = np.append(ecfev1s_tmp, ecfev1_obs)
    print(f"inference took {time.time() - tic} seconds to run")

    # Add to df
    df.loc[df.ID == id, "Airway Resistance mean from ecFEV1 (%)"] = df_for_ID[
        "ecFEV1 inf mean"
    ].values
    df.loc[df.ID == id, "Airway Resistance mean from FEV1 (%)"] = df_for_ID[
        "FEV1 inf mean"
    ].values
    df.loc[df.ID == id, "Airway Resistance mode from ecFEV1 (%)"] = df_for_ID[
        "ecFEV1 inf mode"
    ].values
    df.loc[df.ID == id, "Airway Resistance mode from FEV1 (%)"] = df_for_ID[
        "FEV1 inf mode"
    ].values

# IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
# Happens when the results becomes too close to 0


Running for ID 101, with 816 observations
*** Building FEV1 and O2 point in time model ***


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:310: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(


model took 47.30946373939514 seconds to build
inference took 12.836995840072632 seconds to run

Running for ID 102, with 151 observations
*** Building FEV1 and O2 point in time model ***


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:310: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(


model took 46.9903039932251 seconds to build
inference took 6.473737001419067 seconds to run

Running for ID 103, with 511 observations
*** Building FEV1 and O2 point in time model ***


...

model took 46.97253227233887 seconds to build
inference took 0.2236337661743164 seconds to run

Running for ID 357, with 3 observations
*** Building FEV1 and O2 point in time model ***


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:310: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(


model took 46.93120217323303 seconds to build
inference took 0.5591869354248047 seconds to run

Running for ID 358, with 11 observations
*** Building FEV1 and O2 point in time model ***


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:310: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(


model took 46.92656183242798 seconds to build
inference took 1.7685089111328125 seconds to run


In [9]:
# Check that the computed airway resistance makes sense
# Plot ID 101 FEV1 profile with Date Recorded
# '113', '126', '202', '331'
ID = "101"
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df[df.ID == ID]["Date Recorded"],
        y=df[df.ID == ID]["FEV1"],
        mode="markers",
        name="FEV1",
    )
)
# Add trace for predicted FEV1
# fig.add_trace(go.Scatter(x=df[df.ID == "101"]["Date Recorded"], y=df[df.ID == "101"]["Predicted FEV1"], mode="markers", name="Predicted FEV1"))
# Add trace for airway resistance using s_AW
fig.add_trace(
    go.Scatter(
        x=df[df.ID == ID]["Date Recorded"],
        y=df[df.ID == ID]["Airway Resistance mean from ecFEV1 (%)"],
        mode="markers",
        name="Airway Resistance (%)",
    )
)
# fig.add_trace(go.Scatter(x=df[df.ID == ID]["Date Recorded"], y=s_AW[ID], mode="markers", name="Airway Resistance"))

In [135]:
816 / ((1.78 - 1.23) * 100)
# Compute total amount of measurement divided by max - min for all IDs
df.groupby("ID").apply(
    lambda x: len(x) / max(1, (x["FEV1"].max() - x["FEV1"].min()) * 100)
).sort_values(ascending=False).head(10)

ID
101    14.836364
240    12.423077
286    10.666667
133     9.741935
145     8.629032
203     8.571429
182     8.528302
229     8.480000
149     7.000000
103     5.941860
dtype: float64

### Raw scatter plot

In [142]:
# # 0% airway resistance: FEV1 = Predicted FEV1
# # 25% airway resistance: FEV1 = 0.75 * Predicted FEV1
# # Negative airway resistance: FEV1 > Predicted FEV1
# # Airway resistance = 1 - FEV1/Predicted FEV1
df["Airway Resistance Computed (%)"] = 100 - df["ecFEV1 % Predicted"]

df["Drop from Healthy O2 Saturation (%)"] = (
    df["O2 Saturation"] - df["Healthy O2 Saturation"]
)

df["O2 Saturation % Predicted"] = (
    df["O2 Saturation"] / df["Healthy O2 Saturation"] * 100
)

In [11]:
# Group Airway Resistance Computed (%) into bins of 1% width between 0 and 99
df["Airway Resistance Computed Bins (%)"] = pd.cut(
    df["Airway Resistance Computed (%)"],
    bins=np.arange(0, 100, 1),
    include_lowest=False,
)
df["Airway Resistance Computed Bins Mean (%)"] = df.groupby(
    "Airway Resistance Computed Bins (%)"
)[["Airway Resistance Computed (%)"]].transform("mean")

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_59277/2857819362.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df["Airway Resistance Computed Bins Mean (%)"] = df.groupby(


In [141]:
# Plot Airway resistance vs O2 drop
title = f"O2 Drop vs Airway Resistance computed from ecFEV1 ({df.ID.nunique()} IDs, {len(df)} datapoints)"

# histogram up df["Airway Resistance Computed (%)"] by bins of 1%
# df["Airway Resistance Computed (%)"].hist(bins=100)

fig = px.scatter(
    df,
    x="Airway Resistance Computed (%)",
    # x="Airway Resistance Computed Bins Mean (%)",
    # y="Drop from Healthy O2 Saturation (%)",
    y="O2 Saturation % Predicted",
    title=title,
    hover_data=["ID", "ecFEV1"],
)
# Reduce marker size
fig.update_traces(marker=dict(size=2))
fig.update_layout(font=dict(size=10))
fig.show()
fig.write_image(f"{plotsdir}{title}.pdf")
# 100% is about 98 for females and 97.4 for males
# Hence, threshold at 95% => 3-3.5% drop in O2 Saturation => 96.5-97% O2 Saturation

In [119]:
# Plot Airway resistance vs O2 drop
title = f"O2 Drop vs Airway Resistance from ecFEV1 ({df.ID.nunique()} IDs, {len(df)} datapoints)"
fig = px.scatter(
    df,
    # x="Airway Resistance from ecFEV1 binned (%)",
    x="Airway Resistance mean from FEV1 (%)",
    y="O2 Saturation % Predicted",
    # y="Drop from Healthy O2 Saturation (%)",
    # y="O2 Saturation % FFA",
    title=title,
    hover_data=["ID", "ecFEV1", "FEV1", "O2 Saturation"]
    # customdata=df[['ID', 'ecFEV1']],
    # hovertemplate="<br>".join(
    #     [
    #         "ID: %{customdata[0]}",
    #         "ecFEV1: %{customdata[1]}",
    #         "Airway Resistance from ecFEV1: %{x:.2f}%",
    #         "O2 Saturation: %{y:.2f}%",
    #     ]
    # )
)
# Reduce marker size
fig.update_traces(marker=dict(size=2), opacity=0.3)
fig.show()
fig.write_image(f"{plotsdir}{title}.pdf")
# 100% is about 98 for females and 97.4 for males
# Hence, threshold at 95% => 3-3.5% drop in O2 Saturation => 96.5-97% O2 Saturation

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['ID', 'Date Recorded', 'FEV1', 'O2 Saturation', 'ecFEV1', 'Age', 'Sex', 'Height', 'Predicted FEV1', 'Healthy O2 Saturation', 'ecFEV1 % Predicted', 'FEV1 % Predicted', 'O2 Saturation % Healthy', 'Airway Resistance mean from ecFEV1 (%)', 'Airway Resistance mean from FEV1 (%)', 'Airway Resistance mode from ecFEV1 (%)', 'Airway Resistance mode from FEV1 (%)'] but received: O2 Saturation % Predicted

In [30]:
df.sort_values(by="Airway Resistance mean from ecFEV1 (%)", ascending=True).head(10)

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,...,O2 Saturation % Healthy,Airway Resistance mean from ecFEV1 (%),Airway Resistance mean from FEV1 (%),Airway Resistance mode from ecFEV1 (%),Airway Resistance mode from FEV1 (%),Airway Resistance Computed (%),Drop from Healthy O2 Saturation (%),O2 Saturation % Predicted,Airway Resistance Computed Bins (%),Airway Resistance Computed Bins Mean (%)
2817,113,2021-02-05,5.26,98.0,5.26,26,Male,165.5,3.92364,97.32007,...,100.698654,1.650449,1.650449,0.0,0.0,-34.059204,0.67993,100.698654,NaN,NaN
2818,113,2021-02-08,4.87,99.0,5.26,26,Male,165.5,3.92364,97.32007,...,101.726191,1.650449,2.479764,0.0,0.0,-34.059204,1.67993,101.726191,NaN,NaN
2816,113,2021-02-04,5.18,99.0,5.26,26,Male,165.5,3.92364,97.32007,...,101.726191,1.650449,1.825236,0.0,0.0,-34.059204,1.67993,101.726191,NaN,NaN
2805,113,2021-01-13,5.18,99.0,5.22,26,Male,165.5,3.92364,97.32007,...,101.726191,1.735724,1.825236,0.0,0.0,-33.039743,1.67993,101.726191,NaN,NaN
2803,113,2021-01-11,4.99,98.0,5.22,26,Male,165.5,3.92364,97.32007,...,100.698654,1.735724,2.235904,0.0,0.0,-33.039743,0.67993,100.698654,NaN,NaN
2804,113,2021-01-12,5.22,98.0,5.22,26,Male,165.5,3.92364,97.32007,...,100.698654,1.735724,1.735724,0.0,0.0,-33.039743,0.67993,100.698654,NaN,NaN
2814,113,2021-02-01,4.90,98.0,5.17,26,Male,165.5,3.92364,97.32007,...,100.698654,1.825236,2.354187,0.0,0.0,-31.765416,0.67993,100.698654,NaN,NaN
2815,113,2021-02-02,5.17,99.0,5.18,26,Male,165.5,3.92364,97.32007,...,101.726191,1.825236,1.825236,0.0,0.0,-32.020281,1.67993,101.726191,NaN,NaN
2806,113,2021-01-14,5.10,98.0,5.18,26,Male,165.5,3.92364,97.32007,...,100.698654,1.825236,1.919528,0.0,0.0,-32.020281,0.67993,100.698654,NaN,NaN
2839,113,2021-04-12,4.81,98.0,5.15,26,Male,165.5,3.92364,97.32007,...,100.698654,1.825236,2.613214,0.0,0.0,-31.255685,0.67993,100.698654,NaN,NaN


### Bin AR, fit top envelope

In [139]:
import numpy as np
from scipy.interpolate import BSpline, splrep
from scipy.optimize import curve_fit, minimize


# Group by Airway Resistance and take 80th percentile of O2 Sat / Healthy O2 Sat if there are more than 50 observations
def calc_rmax_o2(df_for_AR, y, percentile=80):
    # print("Taking exact percentile")
    # return np.percentile(df_for_AR[y], percentile), len(df_for_AR)
    # Take data between 80 and 90th percentile
    return (
        np.percentile(
            df_for_AR[y],
            range(percentile - 5, percentile + 5),
        ).mean(),
        len(df_for_AR),
        df_for_AR.ID.nunique(),
    )


# Piecewise fit (constant + polynomial)
def top_envelope_func(x, x0, y0, k1, k2, k3):
    # x0 = 43
    # y0 = df_to_fit[df_to_fit[x] < x0][
    #     o2_col
    # ].mean()
    # y0 = 100.8

    return np.piecewise(
        x,
        [x <= x0],
        [
            lambda x: y0,
            lambda x: k3 * np.power((x - x0), 3)
            + k2 * np.power((x - x0), 2)
            + k1 * (x - x0)
            + y0,
        ],
    )


def fit_factor_profile(df_to_fit, x, y):
    x_data = df_to_fit[x].values
    y_data = df_to_fit[y].values

    def objective(params, x, y):
        return np.sum((top_envelope_func(x, *params) - y) ** 2)

    # Enforce monotonously decreasing function
    constraints = {
        "type": "ineq",
        "fun": lambda params: np.diff(-top_envelope_func(x_data, *params)),
    }
    # constraints = ()

    # Initial guess for parameters
    # initial_guess = np.array([ 37, 101, -2.27528544e-01, 1.85798417e-02, -5.24480408e-04])
    initial_guess = np.array([37, 101, 0, 0, 0])
    # # Minimize the objective function with the constraint
    result = minimize(
        objective, initial_guess, args=(x_data, y_data), constraints=constraints
    )
    parameters = result.x

    # parameters, covariance = curve_fit(
    #     top_envelope_func,
    #     df_to_fit[x].values,
    #     df_to_fit[y].values,
    # )
    print(f"Parameters: {parameters}")
    df_to_fit["Piecewise fit"] = top_envelope_func(x_data, *parameters)

    # Spline fit
    ## Base value for smoothing parameter
    s = df_to_fit.shape[0] - np.sqrt(2 * df_to_fit.shape[0])
    # print(f"Smoothing parameter: {s}")
    ### Create a spline representation of the curve
    ### tck-tuple: (t,c,k) containing the vector of knots, the B-spline coefficients, and the degree of the spline.
    tck = splrep(
        x_data,
        y_data,
        s=s,
    )
    ### Evalute the spline repr on a new set of points
    df_to_fit["Spline"] = BSpline(*tck)(df_to_fit[x])

    # Mean smoothing
    df_to_fit["Mean Smoothing"] = df_to_fit[y].rolling(5, center=True).mean()
    return df_to_fit

In [140]:
fev1_col = "FEV1"
fev1_col = "ecFEV1"

ar_col = f"Airway Resistance mean from {fev1_col} (%)"

df[f"Airway Resistance mean from {fev1_col} binned (%)"] = pd.cut(
    df[f"Airway Resistance mean from {fev1_col} (%)"],
    bins=np.arange(0, 100, 2),
    labels=False,
)
# df["Airway Resistance from ecFEV1 binned (%)"] = df["Airway Resistance from ecFEV1 binned (%)"].astype(str)
df[f"Airway Resistance mean from {fev1_col} binned (%)"] = (
    df[f"Airway Resistance mean from {fev1_col} binned (%)"] * 2.0
)
# Order df by values of Airway Resistance from ecFEV1 binned (%)
# df.sort_values(by="Airway Resistance from ecFEV1 binned (%)")

In [143]:
df["O2 Saturation % Predicted"] = (
    df["O2 Saturation"] / df["Healthy O2 Saturation"] * 100
)

ar_col_binned = f"Airway Resistance mean from {fev1_col} binned (%)"

df_for_AW_O2Sat_study = df.copy()
df_for_AW_O2Sat_study = df[~df.ID.isin(["122"])].copy()
# df_for_AW_O2Sat_study = df[~df.ID.isin(["122", "286"])].copy()

for prctile in [10, 85, 90]:  # range(60, 90, 5):
    # for prctile in [90]:  # range(60, 90, 5):
    rmax_o2_sat_col = f"{prctile}th-rmax O2 Saturation<br> % Predicted"

    rmax_AW_O2Sat = df_for_AW_O2Sat_study.groupby([ar_col_binned]).apply(
        lambda x: calc_rmax_o2(x, "O2 Saturation % Predicted", prctile)
    )
    # Unstack rmax_AR_O2Sat tuples into 2 columns
    rmax_AW_O2Sat = (
        rmax_AW_O2Sat.apply(pd.Series)
        .rename(columns={0: rmax_o2_sat_col, 1: "#datapoints", 2: "#IDs"})
        .reset_index()
    )
    # Add column for >50 datapoints
    rmax_AW_O2Sat[">50 datapoints"] = rmax_AW_O2Sat["#datapoints"] >= 50
    rmax_AW_O2Sat[">10 IDs"] = rmax_AW_O2Sat["#IDs"] >= 10

    # Mask for >50 datapoints
    rmax_AW_O2Sat_plot = fit_factor_profile(
        # rmax_AW_O2Sat[rmax_AW_O2Sat[">50 datapoints"]].copy(),
        rmax_AW_O2Sat[
            rmax_AW_O2Sat[">50 datapoints"] & rmax_AW_O2Sat[">10 IDs"]
        ].copy(),
        x=ar_col_binned,
        y=rmax_o2_sat_col,
    )

    # PLot rmax_AW_O2Sat
    title = f"Max achievable O2 Saturation % Predicted ({prctile}th-rmax) vs Airway Resistance ({df_for_AW_O2Sat_study.ID.nunique()} IDs, {len(df_for_AW_O2Sat_study)} datapoints)"
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=rmax_AW_O2Sat_plot[ar_col_binned],
            y=rmax_AW_O2Sat_plot[rmax_o2_sat_col],
            mode="markers",
            name="Airway Resistance vs O2 drop",
        ),
    )
    # fig.add_trace(
    #     go.Scatter(
    #         x=rmax_AW_O2Sat_plot[ar_col_binned],
    #         y=rmax_AW_O2Sat_plot["Spline"],
    #         mode="lines",
    #         name="Spline",
    #     )
    # )
    fig.add_trace(
        go.Scatter(
            x=rmax_AW_O2Sat_plot[ar_col_binned],
            y=rmax_AW_O2Sat_plot["Piecewise fit"],
            mode="lines",
            name="Constant + 3rd order polynomial fit",
        ),
    )
    fig.update_traces(line=dict(width=1), marker=dict(size=3))
    fig.update_yaxes(
        range=[90, 103.5],
        nticks=10,
        title=rmax_o2_sat_col,
    )
    fig.update_xaxes(title=ar_col_binned, range=[-5, 91], nticks=10)
    fig.update_layout(title=title, height=300, width=800, font=dict(size=8))
    fig.show()
    # Save to file
    fig.write_image(
        f"{plotsdir}{title}.pdf",
        width=1000,
        height=400,
    )

Parameters: [ 3.38676690e+01  9.81834410e+01 -2.56185853e-01  1.26420777e-02
 -2.53548619e-04]


Parameters: [ 3.45063159e+01  1.00856822e+02 -1.04771757e-01  4.82650316e-03
 -9.88876858e-05]


Parameters: [ 3.46527933e+01  1.00959840e+02 -6.42530178e-02  2.42057975e-03
 -5.42896723e-05]


In [63]:
# Bar plot of number of datapoints per airway resistance bin
title = f"Number of datapoints per Airway Resistance bin ({df_for_AW_O2Sat_study.ID.nunique()} IDs, {len(df_for_AW_O2Sat_study)} datapoints)"
fig = go.Figure()

# rmax_AW_O2Sat[rmax_AW_O2Sat[">50 datapoints"] & rmax_AW_O2Sat[">5 IDs"]]

fig.add_trace(
    go.Bar(
        x=rmax_AW_O2Sat[ar_col_binned],
        y=rmax_AW_O2Sat[rmax_AW_O2Sat[">50 datapoints"]]["#datapoints"].astype(int),
        name="#datapoints",
        marker=dict(color="grey"),
    ),
)
# Add line for 50 datapoints
fig.add_trace(
    go.Scatter(
        x=rmax_AW_O2Sat[ar_col_binned],
        y=np.repeat(50, len(rmax_AW_O2Sat)),
        mode="lines",
        name="50 datapoints",
    )
)
# Log y axis
fig.update_yaxes(title="#datapoints", nticks=2)
fig.update_yaxes(type="log", title="#datapoints", nticks=2)
# Range x axis to 91
fig.update_xaxes(range=[-5, 91], nticks=10, title="Binned Airway Resistance (%)")
fig.update_layout(font=dict(size=8), height=250, width=800, title=title)
fig.show()

In [64]:
rmax_AW_O2Sat[rmax_AW_O2Sat[">50 datapoints"]]

,Airway Resistance mean from ecFEV1 binned (%),90th-rmax O2 Saturation<br> % Predicted,#datapoints,#IDs,>50 datapoints,>5 IDs
1,2.0,101.726191,92.0,6.0,True,True
2,4.0,100.815880,636.0,11.0,True,True
3,6.0,100.948323,558.0,20.0,True,True
4,8.0,101.081507,546.0,26.0,True,True
5,10.0,101.200835,669.0,35.0,True,True
6,12.0,101.112838,662.0,46.0,True,True
7,14.0,101.412914,649.0,52.0,True,True
8,16.0,100.896248,575.0,48.0,True,True
9,18.0,100.887696,526.0,43.0,True,True
10,20.0,100.883963,538.0,49.0,True,True


In [31]:
rmax_AW_O2Sat.sort_values(by="#datapoints", ascending=False).head(10)
rmax_AW_O2Sat_plot.sort_values(
    by="Airway Resistance mean from ecFEV1 binned (%)", ascending=False
).head(10)

,Airway Resistance from ecFEV1 binned (%),80th-rmax O2 Saturation<br> % Predicted,#datapoints,>50 datapoints,Piecewise fit,Spline,Mean Smoothing
76,78,93.875626,64.0,True,94,95.796427,NaN
69,71,98.602302,87.0,True,97,97.510230,NaN
66,68,98.889212,62.0,True,98,98.095555,98.002900
65,67,98.822189,237.0,True,98,98.272424,99.061898
63,65,99.825172,223.0,True,99,98.600152,98.940245
62,64,99.170613,221.0,True,99,98.751460,99.120647
61,63,97.994039,83.0,True,99,98.894698,99.188575
59,61,99.791221,736.0,True,99,99.157860,99.285950
58,60,99.161829,116.0,True,99,99.278235,99.405346
57,59,100.312046,265.0,True,99,99.391438,99.835555


### Specific cases for the plot with Airway resistance computed

In [17]:
# Plot FEV1 % Predicted with time for individual 122
def plot_fev1_o2(df, ids, lf_col="ecFEV1 % Predicted", o2_col="O2 Saturation"):
    for id in ids:
        df_for_ID = df[df.ID == id]
        # Create subplot with 2 rows
        fig = make_subplots(rows=2, cols=1)
        # Add trace for FEV1 % Predicted on one subplot
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID[lf_col],
                mode="markers",
                name=lf_col,
            ),
            row=1,
            col=1,
        )
        # fig.add_trace(
        #     go.Scatter(
        #         x=df_for_ID["Date Recorded"],
        #         y=df_for_ID["FEV1 % Predicted"],
        #         mode="markers",
        #         name="FEV1 % Predicted",
        #     ),
        #     row=1,
        #     col=1,
        # )
        # Add trace for O2 Saturation on another subplot
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID[o2_col],
                mode="markers",
                name="O2 Saturation",
            ),
            row=2,
            col=1,
        )
        fig.update_traces(marker=dict(size=3), line=dict(width=0.5))
        title = (
            f"{lf_col} and {o2_col} for individual {id} ({len(df_for_ID)} datapoints)"
        )
        fig.update_layout(title=title)
        # Add trace for O2 Saturation on another subplot

        fig.show()

#### Specific individuals

In [18]:
# ID 286
plot_fev1_o2(df, ["286"], lf_col="FEV1", o2_col="O2 Saturation")

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [114]:
# Save this to excel
tmp = df.groupby(["ID", "FEV1"]).count()["Date Recorded"]
# Within each ID, sort by count
tmp = tmp.groupby(level=0).apply(lambda x: x.sort_values(ascending=False))
# Remove second ID from index
tmp.index = tmp.index.droplevel(1)
# Keep only first 5
tmp = tmp.groupby(level=0).head(5)
# Get mask of the first value for each ID
ids_idx = tmp.groupby(level=0).head(1).index
# Order rows by this mask
ids_list = tmp.loc[ids_idx]
# Sort by Date Recorded
ids_list = ids_list.sort_values(ascending=False)
# Get IDs list
ids_list = ids_list.reset_index()["ID"]
# Sort the values in the index in order of the IDs list
tmp = tmp.reset_index()
# Sort by IDs list
tmp = tmp.set_index("ID").loc[ids_list]
# Set multi index ID and FEV1
tmp = tmp.reset_index().set_index(["ID", "FEV1"])
# Save to excel
tmp.to_excel(f"{plotsdir}FEV1_counts.xlsx")

In [116]:
df.groupby(["ID", "FEV1", "Date Recorded"]).count()["Age"].sort_values(
    ascending=False
).head(10)

ID   FEV1  Date Recorded
101  1.23  2019-10-28       1
201  2.03  2021-03-31       1
     2.07  2020-10-18       1
           2020-10-12       1
     2.06  2021-06-21       1
           2021-05-14       1
           2021-03-20       1
           2021-01-02       1
     2.04  2021-02-21       1
     2.03  2020-11-07       1
Name: Age, dtype: int64

#### Low airway resistance

In [ ]:
# Filter airway resistance below 40%
df[df["Airway Resistance (%)"] < -20]
# '113', '126', '202', '331'

In [20]:
plot_fev1_o2(df, ["113", "126", "202", "331"], lf_col="FEV1", o2_col="O2 Saturation")

#### High airway resistance

In [ ]:
# Filter airway resistance below 40%
df[df["Airway Resistance (%)"] > 80].ID.unique()
# 3 individuals '122', '198', '286' have airway resistance > 80%

In [165]:
# Plot FEV1 % Predicted with time for individual 122
plot_fev1_o2(df, ["122", "286"])

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



#### High O2 Drop

In [9]:
df[df["Drop from Healthy O2 Saturation (%)"] < 90].ID.unique()

KeyError: 'Drop from Healthy O2 Saturation (%)'

In [ ]:
plot_fev1_o2(df, ["111", "180", "352"])

#### High positive O2 Drop

In [ ]:
df[df["Drop from Healthy O2 Saturation (%)"] > 2.8].ID.unique()

In [ ]:
df[df.ID == "120"].sort_values(by="Drop from Healthy O2 Saturation (%)")

In [ ]:
plot_fev1_o2(df, ["120"], "Drop from Healthy O2 Saturation (%)")
plot_fev1_o2(df, ["120"])

#### High values (towards 70-72)

In [274]:
ids = ["110", "122", "127", "194", "224", "225"]
plot_fev1_o2(df, ids)

In [79]:
# Amount number of IDs contributing to each bin of Airway Rsistance from ecFEV1 binned (%)
df.groupby("Airway Resistance mean from ecFEV1 binned (%)").ID.unique()[-10:]

Airway Resistance from ecFEV1 binned (%)
69         [112, 127, 155, 224, 253, 258, 282]
70                        [102, 111, 122, 146]
71    [112, 155, 158, 162, 184, 187, 198, 283]
72              [110, 122, 127, 194, 224, 225]
73              [111, 112, 146, 184, 216, 258]
74    [122, 147, 162, 187, 198, 224, 264, 282]
75              [110, 112, 155, 216, 282, 298]
76                             [112, 122, 146]
77                             [110, 122, 198]
78                                       [286]
Name: ID, dtype: object

In [80]:
df.groupby("Airway Resistance mean from ecFEV1 binned (%)").ID.unique()[0:10]

Airway Resistance from ecFEV1 binned (%)
2                                            [113, 202]
3                                  [113, 126, 202, 281]
4                   [113, 126, 177, 202, 250, 281, 331]
5         [113, 121, 126, 177, 230, 250, 262, 281, 331]
6     [113, 121, 126, 177, 230, 250, 281, 288, 302, ...
7     [113, 126, 130, 144, 177, 182, 230, 242, 250, ...
8     [107, 113, 126, 130, 141, 144, 177, 182, 190, ...
9     [113, 121, 126, 130, 144, 152, 182, 190, 233, ...
10    [107, 121, 126, 141, 144, 177, 185, 189, 192, ...
11    [113, 126, 130, 141, 152, 170, 177, 182, 189, ...
Name: ID, dtype: object

### Get drop function

In [56]:
df["Airway Resistance mean from ecFEV1 (%)"].describe()

count    20397.000000
mean        35.114419
std         17.600478
min          1.741280
25%         21.174857
50%         34.714496
75%         49.956804
max         77.629183
Name: Airway Resistance mean from ecFEV1 (%), dtype: float64

In [3]:
# drop_params = np.array(
#     [3.89754850e01, 1.00902396e02, -2.04542149e-01, 1.57422295e-02, -4.00994278e-04]
# )
drop_params = np.array(
    [3.46527933e01, 1.00959840e02, -6.42530178e-02, 2.42057975e-03, -5.42896723e-05]
)

# Update y0 to 0% drop
drop_params[1] = 100
drop_params[2:5] = drop_params[2:5]
# Plote drop function
x = np.arange(0.0, 90.0, 1)
# y = top_envelope_func(x, *drop_params)
# Same using the function from o2satffa
y = o2satffa.multiplicative_drop_func(x) * 100
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode="lines+markers",
        name="Drop function",
    ),
)
fig.update_traces(line=dict(width=1), marker=dict(size=3))
fig.update_yaxes(
    # range=[90, 103.5],
    # nticks=10,
    title="Relative drop (%)",
    # autorange="reversed",
)
fig.update_xaxes(title="Airway Resistance (%)")
fig.update_layout(
    title="Relative drop from Healthy O2 Saturation",
    height=300,
    width=800,
    font=dict(size=8),
)
fig.show()

## F2

In [95]:
# Reduce the Healthy O2 Saturation by the drop amount
df["O2 Saturation FFA (%)"] = df.apply(
    lambda x: x["Healthy O2 Saturation"]
    * top_envelope_func(x["Airway Resistance mean from ecFEV1 (%)"], *drop_params)
    / 100,
    axis=1,
)

In [145]:
df["O2 Saturation % FFA"] = df.apply(
    lambda x: x["O2 Saturation % Predicted"]
    / top_envelope_func(x["Airway Resistance mean from ecFEV1 (%)"], *drop_params)
    * 100,
    axis=1,
)

In [137]:
# Plot O2 Saturation FFA against airway resistance
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df["Airway Resistance mean from ecFEV1 (%)"],
        # y=df["O2 Saturation % Predicted"],
        y=df["O2 Saturation FFA (%)"],
        mode="markers",
        name="O2 Saturation FFA",
    ),
)
fig.show()

In [101]:
df["Drop from O2 Saturation FFA (%)"] = (
    df["O2 Saturation"] / df["O2 Saturation FFA (%)"]
)
df["Drop from O2 Saturation FFA (%)"].describe()

count    20397.000000
mean         0.995453
std          0.015548
min          0.783727
25%          0.988256
50%          0.998497
75%          1.006922
max          1.038079
Name: Drop from O2 Saturation FFA (%), dtype: float64

In [151]:
import src.o2_fev1_analysis.partition as partition

O2_col = "Drop from O2 Saturation FFA (%)"
O2_col = "O2 Saturation % Predicted"
O2_col = "O2 Saturation % FFA"

# # Create 3 equally spaced bins for Airway Resistance
# df["AR group"] = partition.partition_in_n_equal_groups(
#     df["Airway Resistance mean from ecFEV1 (%)"], 5
# )

# Cut Airway Resistance into bins of 0-20, 20-40, 40-60, 60-80
df["AR group"] = pd.cut(
    df["Airway Resistance mean from ecFEV1 (%)"],
    bins=np.arange(0, 100, 20),
    include_lowest=False,
)

group_labels = df["AR group"].unique()
print(f"AR groups: {group_labels}")

# Create subplot with 3 rows
fig = make_subplots(
    rows=len(group_labels) - 1, cols=1, shared_xaxes=True, vertical_spacing=0.02
)
# On first subplot add histogram of Drop from O2 Saturation FFA (%) for 1st AR group
for i in range(len(group_labels) - 1):
    fig.add_trace(
        go.Histogram(
            x=df[df["AR group"] == group_labels[i]][O2_col],
            name=f"Airway Resistance {group_labels[i]}",
            # Bin size of 1
            xbins=dict(start=75, end=110, size=0.2),
        ),
        row=i + 1,
        col=1,
    )

# Smaller font size
# fig.update_xlayout(font=dict(size=10))
# Add title
fig.update_layout(
    title=f"Distribution of {O2_col} for different Airway Resistance groups"
)
# Add x axis to last subplot
fig.update_xaxes(
    title_text="O2 Saturation in % of O2 Saturation if Fully Functional Alveoli",
    row=len(group_labels) - 1,
    col=1,
)

AR groups: [(60.0, 80.0], (40.0, 60.0], (20.0, 40.0], (0.0, 20.0], NaN]
Categories (4, interval[int64, right]): [(0, 20] < (20, 40] < (40, 60] < (60, 80]]


In [120]:
len(group_labels)

5

In [126]:
df

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,...,Airway Resistance Computed (%),Drop from Healthy O2 Saturation (%),O2 Saturation % Predicted,Airway Resistance Computed Bins (%),Airway Resistance Computed Bins Mean (%),Airway Resistance mean from ecFEV1 binned (%),O2 Saturation FFA (%),Drop from O2 Saturation FFA (%),AR group,O2 Saturation % FFA
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.225960,...,63.435523,-0.225960,99.767593,"(63, 64]",63.476089,60.0,96.241939,1.007877,"(60, 80]",100.787662
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.225960,...,63.435523,-1.225960,98.739061,"(63, 64]",63.476089,60.0,96.241939,0.997486,"(60, 80]",99.748614
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.225960,...,63.435523,-1.225960,98.739061,"(63, 64]",63.476089,60.0,96.241939,0.997486,"(60, 80]",99.748614
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.225960,...,63.158519,-0.225960,99.767593,"(63, 64]",63.476089,60.0,96.241939,1.007877,"(60, 80]",100.787662
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.225960,...,62.327508,0.774040,100.796125,"(62, 63]",62.462056,58.0,96.308928,1.017559,"(40, 60]",101.755883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20392,358,2021-06-05,4.33,99.0,4.37,18,Male,177.0,4.505342,97.175768,...,3.004036,1.824232,101.877250,"(3, 4]",3.480643,8.0,97.175768,1.018772,"(0, 20]",101.877250
20393,358,2021-06-09,4.35,99.0,4.35,18,Male,177.0,4.505342,97.175768,...,3.447954,1.824232,101.877250,"(3, 4]",3.480643,8.0,97.175768,1.018772,"(0, 20]",101.877250
20394,358,2021-06-10,4.30,98.0,4.35,18,Male,177.0,4.505342,97.175768,...,3.447954,0.824232,100.848187,"(3, 4]",3.480643,8.0,97.175768,1.008482,"(0, 20]",100.848187
20395,358,2021-06-12,4.30,97.0,4.30,18,Male,177.0,4.505342,97.175768,...,4.557748,-0.175768,99.819123,"(4, 5]",4.538651,8.0,97.175768,0.998191,"(0, 20]",99.819123
